In [ ]:
!pip install google-cloud-pipeline-components

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

PROJECT_ID="dt-tu-sandbox-dev"
BUCKET_NAME="gs://ovo-demos"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8.py310:latest" # Find the right images for tf training and deployment
DEPLOY_IMAGE = "europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest"

PIPELINE_ROOT = f"{BUCKET_NAME}/cnn_pipeline_root/"
MODEL_URI = PIPELINE_ROOT + "tf-model/"

shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

EXPERIMENT_NAME = "cnn-test-experiment"

In [ ]:
import kfp

from kfp import compiler, dsl
from kfp.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics

from google.cloud import aiplatform
from typing import NamedTuple

from kfp.dsl import importer_node
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                          ModelDeployOp)

In [ ]:
@component(base_image=TRAIN_IMAGE, packages_to_install=["tensorflow_datasets==4.6.0"])
def custom_training_task(
    model_uri: str,
    epochs: int = 10,
    batch_size: int = 32,
):
    import tensorflow_datasets as tfds
    import tensorflow as tf
    from tensorflow.python.client import device_lib
    import os
    import sys
    tfds.disable_progress_bar()
    
    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    model_path = model_uri.replace(gs_prefix, gcsfuse_prefix)

    # Load the dataset
    datasets, info = tfds.load('kmnist', with_info=True, as_supervised=True)

    # Normalize and batch process the dataset
    ds_train = datasets['train'].map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).batch(batch_size)


    # Build the Convolutional Neural Network
    model = tf.keras.models.Sequential([                               
          tf.keras.layers.Conv2D(16, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 1), padding = "same"),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(16, (3,3), activation=tf.nn.relu, padding = "same"),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation=tf.nn.relu),
          # TODO: Write the last layer.
          # Hint: KMNIST has 10 output classes.
          tf.keras.layers.Dense(10, activation=tf.nn.softmax),
        ])

    model.compile(optimizer = tf.keras.optimizers.Adam(),
          loss = tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])



    # Train and save the model

    model.fit(ds_train, epochs=epochs)

    # TODO: Save your CNN classifier. 
    tf.saved_model.save(model,model_path)



In [ ]:
@pipeline(
    name="cnn-solution",
    description="cnn training pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(
    model_display_name: str, 
    model_uri:str, 
    epochs: int, 
    batch_size: int,
    model_deploy_compute: str
):
    training_task = custom_training_task(
        model_uri=model_uri,
        epochs=epochs,
        batch_size=batch_size
    )
    
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=model_uri,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": DEPLOY_IMAGE 
            },
        },
    ).after(training_task)

    model_upload_op = ModelUploadOp(
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    endpoint_create_op = EndpointCreateOp(
        display_name="cnn-pipeline-created-endpoint",
    )
    
    _ = ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type=model_deploy_compute,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="cnn-pipeline.json"
)

In [ ]:
job = aiplatform.PipelineJob(
    display_name=f"cnn-pipeline-{TIMESTAMP}",
    template_path="cnn-pipeline.json",
    enable_caching=True,
    parameter_values={
        "epochs":1,
        "batch_size":32,
        "model_uri":MODEL_URI,
        "model_display_name":f"cnn-pipeline-model-{TIMESTAMPT}",
        "model_deploy_compute":"n1-standard-4"
    }
)

In [ ]:
job.submit(service_account=SERVICE_ACCOUNT, experiment=EXPERIMENT_NAME)